In [11]:
import pandas as pd
import osmnx as ox
import tensorflow as tf
import numpy as np
import random
from collections import deque
import osmnx as ox
import networkx as nx
import folium
from matplotlib import cm
import matplotlib.colors as mcolors

In [189]:
# # Define depot and customer locations
# locations = [
#     (-7.266735,112.736115),  # Depot
#     (-7.291418,112.677844)   # Customer 1
# ]
# depot_location = locations[0]

# # Fetch road network with extended distance
# G = ox.graph_from_point(depot_location, dist=15000, network_type='drive')

# # Function to validate nearest nodes and map locations to the road network
# def map_to_nearest_nodes(locations, graph, distance_threshold=2000):
#     valid_nodes = []
#     valid_demands = []
#     for idx, (lat, lng) in enumerate(locations):
#         nearest_node = ox.nearest_nodes(graph, lng, lat)
#         nearest_point = (graph.nodes[nearest_node]['y'], graph.nodes[nearest_node]['x'])
#         dist_to_nearest = ox.distance.great_circle(lat, lng, nearest_point[0], nearest_point[1])
#         print(f"Location {idx}: Distance to nearest node = {dist_to_nearest:.2f} meters")
#         if dist_to_nearest > distance_threshold:  # Skip locations too far from the network
#             print(f"Warning: Location {idx} is far from the road network! Skipping this location.")
#             continue
#         valid_nodes.append(nearest_node)
#         if idx > 0:  # Exclude depot from demands
#             valid_demands.append(10)
#     return valid_nodes, valid_demands

# # Map locations to nearest nodes
# valid_nodes, valid_demands = map_to_nearest_nodes(locations, G)

# # Convert graph to undirected for connectivity check
# G_undirected = G.to_undirected()

# # Check connectivity of the graph
# if not nx.is_connected(G_undirected):
#     print("Warning: The road network graph is not fully connected. Consider increasing the distance or changing locations.")
# else:
#     print("The road network graph is fully connected.")

# # Build distance matrix for valid nodes
# def build_distance_matrix(valid_nodes, graph):
#     distance_matrix = np.zeros((len(valid_nodes), len(valid_nodes)))
#     paths = {}
#     for i, start_node in enumerate(valid_nodes):
#         for j, end_node in enumerate(valid_nodes):
#             if i != j:
#                 try:
#                     path = nx.shortest_path(graph, start_node, end_node, weight='length')
#                     distance = nx.shortest_path_length(graph, start_node, end_node, weight='length')
#                     distance_matrix[i][j] = distance
#                     paths[(i, j)] = path
#                 except nx.NetworkXNoPath:
#                     print(f"Warning: No path between nodes {start_node} and {end_node}")
#                     distance_matrix[i][j] = float('inf')
#                     paths[(i, j)] = []
#     return distance_matrix, paths

In [ ]:
# Dataset baru: Jalur dari Stasiun Gubeng ke tunjungan plaza
routes_data = [
    {
        "Condition": "lancar",
        "Route": [(-7.257472, 112.752088), (-7.261500, 112.744000), (-7.266735, 112.736115)],
        "Time": 10,  # in minutes
        "Distance": 3.5  # in km
    },
    {
        "Condition": "macet",
        "Route": [(-7.257472, 112.752088), (-7.263000, 112.740000), (-7.266735, 112.736115)],
        "Time": 20,  # in minutes
        "Distance": 4.0  # in km
    }
]

# Fetch road network with extended distance
start_location = routes_data[0]["Route"][0]
G = ox.graph_from_point(start_location, dist=15000, network_type='drive')

# Map routes to nearest nodes and find actual paths on the road network
def map_and_find_paths(routes, graph):
    mapped_routes = []
    for route in routes:
        nodes = []
        for lat, lng in route["Route"]:
            nearest_node = ox.nearest_nodes(graph, lng, lat)
            nodes.append(nearest_node)

        # Find shortest path between consecutive nodes
        path_coords = []
        for i in range(len(nodes) - 1):
            try:
                # Get the shortest path (list of nodes)
                path = nx.shortest_path(graph, nodes[i], nodes[i + 1], weight="length")
                # Convert path nodes to coordinates
                path_coords.extend([(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in path])
            except nx.NetworkXNoPath:
                print(f"Warning: No path between {nodes[i]} and {nodes[i + 1]} in {route['Condition']} route.")
        
        mapped_routes.append({
            "Condition": route["Condition"],
            "Path": path_coords,
            "Time": route["Time"],
            "Distance": route["Distance"]
        })
    return mapped_routes

# Map and calculate paths for all routes
mapped_routes_with_paths = map_and_find_paths(routes_data, G)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/osmnx/graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(


In [191]:
# # Define DQN model
# class DQN(tf.keras.Model):
#     def __init__(self, num_actions):
#         super(DQN, self).__init__()
#         self.fc1 = tf.keras.layers.Dense(256, activation='relu')
#         self.fc2 = tf.keras.layers.Dense(256, activation='relu')
#         self.fc3 = tf.keras.layers.Dense(128, activation='relu')
#         self.output_layer = tf.keras.layers.Dense(num_actions, activation=None)

#     def call(self, inputs):
#         x = self.fc1(inputs)
#         x = self.fc2(x)
#         x = self.fc3(x)
#         return self.output_layer(x)

In [39]:
# Define DQN model
class DQN(tf.keras.Model):
    def __init__(self, num_actions):
        super(DQN, self).__init__()
        self.fc1 = tf.keras.layers.Dense(256, activation='relu')
        self.fc2 = tf.keras.layers.Dense(256, activation='relu')
        self.fc3 = tf.keras.layers.Dense(128, activation='relu')
        self.output_layer = tf.keras.layers.Dense(num_actions, activation=None)

    def call(self, inputs):
        x = self.fc1(inputs)
        x = self.fc2(x)
        x = self.fc3(x)
        return self.output_layer(x)

In [6]:
# class ReplayBuffer:
#     def __init__(self, buffer_size=10000):
#         self.buffer = deque(maxlen=buffer_size)

#     def add(self, experience):
#         self.buffer.append(experience)

#     def sample(self, batch_size):
#         return random.sample(self.buffer, batch_size)

#     def __len__(self):
#         return len(self.buffer)

In [40]:
class ReplayBuffer:
    def __init__(self, buffer_size=10000):
        self.buffer = deque(maxlen=buffer_size)

    def add(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def __len__(self):
        return len(self.buffer)

In [196]:
# # Define VRP Agent
# class VRPAgent:
#     def __init__(self, num_actions, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, learning_rate=0.001):
#         self.num_actions = num_actions
#         self.gamma = gamma
#         self.epsilon = epsilon
#         self.epsilon_decay = epsilon_decay
#         self.epsilon_min = epsilon_min
#         self.model = DQN(num_actions)
#         self.target_model = DQN(num_actions)
#         self.optimizer = tf.keras.optimizers.Adam(learning_rate)
#         self.replay_buffer = ReplayBuffer()
#         self.update_target_model()

#     def update_target_model(self):
#         self.target_model.set_weights(self.model.get_weights())

#     def choose_action(self, state, unvisited_nodes):
#         state_tensor = tf.convert_to_tensor([[state]], dtype=tf.float32)
#         if np.random.rand() < self.epsilon:
#             return np.random.choice(list(unvisited_nodes))
#         else:
#             q_values = self.model(state_tensor).numpy()[0]
#             masked_q_values = np.full_like(q_values, -np.inf)
#             masked_q_values[list(unvisited_nodes)] = q_values[list(unvisited_nodes)]
#             return np.argmax(masked_q_values)

#     def store_experience(self, state, action, reward, next_state, done):
#         state = np.atleast_2d(state)
#         next_state = np.atleast_2d(next_state)
#         self.replay_buffer.add((state, action, reward, next_state, done))

#     def train(self, batch_size=32):
#         if len(self.replay_buffer) < batch_size:
#             return
#         batch = self.replay_buffer.sample(batch_size)
#         states, actions, rewards, next_states, dones = zip(*batch)
#         states = tf.convert_to_tensor(np.array(states).squeeze(axis=1), dtype=tf.float32)
#         next_states = tf.convert_to_tensor(np.array(next_states).squeeze(axis=1), dtype=tf.float32)
#         rewards = tf.convert_to_tensor(rewards, dtype=tf.float32)
#         actions = tf.convert_to_tensor(actions, dtype=tf.int32)
#         dones = tf.convert_to_tensor(dones, dtype=tf.float32)
#         next_q_values = self.target_model(next_states)
#         max_next_q_values = tf.reduce_max(next_q_values, axis=1)
#         targets = rewards + (1 - dones) * self.gamma * max_next_q_values
#         with tf.GradientTape() as tape:
#             q_values = self.model(states)
#             action_masks = tf.one_hot(actions, self.num_actions)
#             q_values = tf.reduce_sum(action_masks * q_values, axis=1)
#             loss = tf.keras.losses.MSE(targets, q_values)
#         gradients = tape.gradient(loss, self.model.trainable_variables)
#         self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
#         if self.epsilon > self.epsilon_min:
#             self.epsilon *= self.epsilon_decay

In [41]:
# Define VRP Agent
class VRPAgent:
    def __init__(self, num_actions, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, learning_rate=0.001):
        self.num_actions = num_actions
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.model = DQN(num_actions)
        self.target_model = DQN(num_actions)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate)
        self.replay_buffer = ReplayBuffer()
        self.update_target_model()

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def choose_action(self, state):
        state_tensor = tf.convert_to_tensor([state], dtype=tf.float32)
        if np.random.rand() < self.epsilon:
            return random.choice(range(self.num_actions))
        else:
            q_values = self.model(state_tensor).numpy()[0]
            return np.argmax(q_values)

    def store_experience(self, state, action, reward, next_state, done):
        state = np.atleast_2d(state)
        next_state = np.atleast_2d(next_state)
        self.replay_buffer.add((state, action, reward, next_state, done))

    def train(self, batch_size=32):
        if len(self.replay_buffer) < batch_size:
            return
        batch = self.replay_buffer.sample(batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        states = tf.convert_to_tensor(np.array(states).squeeze(axis=1), dtype=tf.float32)
        next_states = tf.convert_to_tensor(np.array(next_states).squeeze(axis=1), dtype=tf.float32)
        rewards = tf.convert_to_tensor(rewards, dtype=tf.float32)
        actions = tf.convert_to_tensor(actions, dtype=tf.int32)
        dones = tf.convert_to_tensor(dones, dtype=tf.float32)
        next_q_values = self.target_model(next_states)
        max_next_q_values = tf.reduce_max(next_q_values, axis=1)
        targets = rewards + (1 - dones) * self.gamma * max_next_q_values
        with tf.GradientTape() as tape:
            q_values = self.model(states)
            action_masks = tf.one_hot(actions, self.num_actions)
            q_values = tf.reduce_sum(action_masks * q_values, axis=1)
            loss = tf.keras.losses.MSE(targets, q_values)
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [198]:
# def train_vrp_agent(agent, num_vehicles, capacity, customer_demands, distance_matrix, num_episodes=150, batch_size=32, update_target_every=10):
#     best_routes = []
#     best_total_distance = float('inf')

#     for episode in range(num_episodes):
#         total_distance = 0
#         cumulative_reward = 0
#         episode_routes = []
#         unvisited_nodes = set(range(1, len(customer_demands) + 1))  # Adjusted for valid_nodes (exclude depot)

#         for vehicle_idx in range(num_vehicles):
#             route = [0]  # Start from depot (node index 0)
#             load = 0
#             current_node = 0

#             while unvisited_nodes:
#                 action = agent.choose_action(current_node, unvisited_nodes)

#                 if action in unvisited_nodes:
#                     dist = distance_matrix[current_node][action]
#                     if load + customer_demands[action - 1] <= capacity:
#                         route.append(action)
#                         load += customer_demands[action - 1]
#                         total_distance += dist
#                         unvisited_nodes.remove(action)

#                         reward = max(10 / (dist + 1e-5), 1)
#                         reward += 10

#                         cumulative_reward += reward
#                         agent.store_experience(np.array([current_node]), action, reward, np.array([action]), False)

#                         current_node = action
#                     else:
#                         reward = -100
#                         agent.store_experience(np.array([current_node]), action, reward, np.array([current_node]), True)
#                         break
#                 else:
#                     reward = -50
#                     agent.store_experience(np.array([current_node]), action, reward, np.array([current_node]), True)
#                     break

#             if unvisited_nodes or vehicle_idx < num_vehicles - 1:
#                 route.append(0)
#                 total_distance += distance_matrix[current_node][0]
#                 reward = 100
#                 cumulative_reward += reward
#             elif vehicle_idx == num_vehicles - 1:
#                 print(f"Last route ends at node {current_node} without returning to depot.")

#             episode_routes.append(route)

#         if total_distance < best_total_distance:
#             best_total_distance = total_distance
#             best_routes = episode_routes

#         print(f"Episode {episode + 1}/{num_episodes} - Total Distance: {total_distance} - Reward: {cumulative_reward:.2f} - Epsilon: {agent.epsilon:.4f}")

#         agent.train(batch_size=batch_size)
#         if episode % update_target_every == 0:
#             agent.update_target_model()

#     print(f"Best Total Distance: {best_total_distance}")
#     return best_routes


In [42]:
# Train the agent to choose between routes
agent = VRPAgent(num_actions=2)
state = np.array([route["Time"] for route in routes_data], dtype=np.float32)  # Use times from routes
num_episodes = 100

for episode in range(num_episodes):
    action = agent.choose_action(state)
    reward = -state[action]  # Minimize time
    next_state = state  # State remains constant
    agent.store_experience(state, action, reward, next_state, True)
    agent.train(batch_size=32)
    print(f"Episode {episode + 1}: Chosen action = {action}, Reward = {reward}")

# Get the best route
best_action = agent.choose_action(state)
print(f"The best route is '{mapped_routes_with_paths[best_action]['Condition']}' with time {state[best_action]} minutes.")


Episode 1: Chosen action = 0, Reward = -10.0
Episode 2: Chosen action = 0, Reward = -10.0
Episode 3: Chosen action = 0, Reward = -10.0
Episode 4: Chosen action = 1, Reward = -20.0
Episode 5: Chosen action = 0, Reward = -10.0
Episode 6: Chosen action = 0, Reward = -10.0
Episode 7: Chosen action = 1, Reward = -20.0
Episode 8: Chosen action = 0, Reward = -10.0
Episode 9: Chosen action = 0, Reward = -10.0
Episode 10: Chosen action = 1, Reward = -20.0
Episode 11: Chosen action = 0, Reward = -10.0
Episode 12: Chosen action = 1, Reward = -20.0
Episode 13: Chosen action = 1, Reward = -20.0
Episode 14: Chosen action = 0, Reward = -10.0
Episode 15: Chosen action = 1, Reward = -20.0
Episode 16: Chosen action = 0, Reward = -10.0
Episode 17: Chosen action = 0, Reward = -10.0
Episode 18: Chosen action = 0, Reward = -10.0
Episode 19: Chosen action = 1, Reward = -20.0
Episode 20: Chosen action = 0, Reward = -10.0
Episode 21: Chosen action = 1, Reward = -20.0
Episode 22: Chosen action = 1, Reward = -20

In [201]:
# # Initialize and train the VRP Agent
# capacity = 100
# num_vehicles = 1
# # Initialize and train the VRP Agent
# num_actions = len(distance_matrix[0])
# agent = VRPAgent(num_actions=num_actions)
# routes = train_vrp_agent(agent, num_vehicles=1, capacity=100, customer_demands=valid_demands, distance_matrix=distance_matrix)


In [ ]:
# Visualization with folium
start_location = routes_data[0]["Route"][0]
m = folium.Map(location=start_location, zoom_start=14)

# Add start and end markers
folium.Marker(
    location=start_location,
    popup="Start: Stasiun Gubeng",
    icon=folium.Icon(color="green")
).add_to(m)

end_location = routes_data[0]["Route"][-1]
folium.Marker(
    location=end_location,
    popup="End: Tunjungan plaza",
    icon=folium.Icon(color="red")
).add_to(m)

# Color palette for routes
color_palette = cm.get_cmap('rainbow', len(mapped_routes_with_paths))

# Add routes to the map
for i, route in enumerate(mapped_routes_with_paths):
    route_color = mcolors.to_hex(color_palette(i / len(mapped_routes_with_paths)))
    folium.PolyLine(
        locations=route["Path"],
        color=route_color,
        weight=5,
        opacity=0.8,
        tooltip=f"Condition: {route['Condition']} | Time: {route['Time']} mins | Distance: {route['Distance']} km"
    ).add_to(m)

# Save the map
m.save("vrp_routes_tunjungank.html")
print("Routes visualized in 'vrp_routes_with_network.html'")


Routes visualized in 'vrp_routes_with_network.html'


/var/folders/bd/1gfn4wvd2yj_rjzsdjyq1g940000gn/T/ipykernel_60688/2868313022.py:20: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  color_palette = cm.get_cmap('rainbow', len(mapped_routes_with_paths))
